### Mecab Colab에서 사용하기 위한 과정, Komoran이기 때문에 현재 필요는 없습니다.

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

###### 0. Importing KKMa, Komoran, Hannaum, Okt

In [ ]:
import konlpy
import pandas as pd
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

In [ ]:
from google.colab import files
myfile = files.upload()

Saving Naver_Webtoon_Crawling.csv to Naver_Webtoon_Crawling.csv


###### 1. Data Cleaned

In [ ]:
def preprocessing_data(webtoon):
  webtoon.drop(['Unnamed: 0','author','day'], axis = 1, inplace = True)
  webtoon['title_genre_story'] = webtoon['title'] + " " + webtoon['genre'] + " " + webtoon['story']
  return webtoon 

#데이터 전처리 방법

In [ ]:
webtoon = pd.read_csv('Naver_Webtoon_Crawling.csv')
webtoon.head()

,Unnamed: 0,title,author,day,genre,story
0,0,참교육,채용택 / 한가람,월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...
1,1,신의 탑,SIU,월,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑
2,2,뷰티풀 군바리,설이 / 윤성원,월,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!
3,3,소녀의 세계,모랑지,월,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나\n여러 갈등을 함께 ...
4,4,팔이피플,매미 / 희세,월,드라마,"&lt;마스크걸&gt;, &lt;위대한 방옥숙&gt; 매미/희세 작가의 신작!\nS..."


In [ ]:
webtoon['story'] = webtoon['story'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
webtoon.head()

#story 부분 xss 막기위해 처리된 부분, \n 부분 지워주기 위한 정규화

,Unnamed: 0,title,author,day,genre,story
0,0,참교육,채용택 / 한가람,월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부활남 채용택 ...
1,1,신의 탑,SIU,월,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑
2,2,뷰티풀 군바리,설이 / 윤성원,월,드라마,여자도 군대에 간다면본격 여자도 군대 가는 만화
3,3,소녀의 세계,모랑지,월,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여러 갈등을 함께 겪으...
4,4,팔이피플,매미 / 희세,월,드라마,마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범한 유부녀 박주연...


In [ ]:
preprocessing_data(webtoon)
webtoon.head()

,title,genre,story,title_genre_story
0,참교육,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부활남 채용택 ...,참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부...
1,신의 탑,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑,신의 탑 판타지 자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년...
2,뷰티풀 군바리,드라마,여자도 군대에 간다면본격 여자도 군대 가는 만화,뷰티풀 군바리 드라마 여자도 군대에 간다면본격 여자도 군대 가는 만화
3,소녀의 세계,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여러 갈등을 함께 겪으...,소녀의 세계 드라마 완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여...
4,팔이피플,드라마,마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범한 유부녀 박주연...,팔이피플 드라마 마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범...


###### 2. 형태소 분석 과정

In [ ]:
#stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

komoran = Komoran()
td = []
for sentence in webtoon['title_genre_story']:
  temp_X = komoran.nouns(sentence)
  #temp_X = [word for word in temp_X if not word in stopwords]
  td.append(temp_X)

webtoon['komoran'] = td

###### 3. Result

In [ ]:
webtoon.head()

,title,genre,story,title_genre_story,komoran
0,참교육,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부활남 채용택 ...,참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부...,"[참, 교육, 액션, 교권, 교권, 보호국, 소속, 진의, 참, 교육, 시작, 다,..."
1,신의 탑,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑,신의 탑 판타지 자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년...,"[신의 탑, 판타지, 자신, 것, 소녀, 탑, 소년, 소년, 시험, 탑]"
2,뷰티풀 군바리,드라마,여자도 군대에 간다면본격 여자도 군대 가는 만화,뷰티풀 군바리 드라마 여자도 군대에 간다면본격 여자도 군대 가는 만화,"[뷰티풀, 군바리, 드라마, 여자도, 군대, 간다, 면, 본격, 여자도, 군대, 만화]"
3,소녀의 세계,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여러 갈등을 함께 겪으...,소녀의 세계 드라마 완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여...,"[소녀, 세계, 드라마, 완벽, 사실, 백조, 맘, 씨, 오리, 만나, 갈등, 친구..."
4,팔이피플,드라마,마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범한 유부녀 박주연...,팔이피플 드라마 마스크걸 위대한 방옥숙 매미희세 작가의 신작에서 육아용품 파는 평범...,"[팔이, 피플, 드라마, 마스크, 거, 방, 옥, 숙, 매미, 작가, 신작, 육아,..."


In [ ]:
Webtoon_cleaned = pd.DataFrame(webtoon)

In [ ]:
Webtoon_cleaned.to_csv('Webtoon_cleaned.csv')
#!cat webtoon_processing.csv

In [ ]:
files.download('Webtoon_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>